### Installing Dependencies

In [2]:
!pip install cplex
!pip install docplex
from docplex.mp.model import Model

### Defining the Model

In [651]:
energyModel = Model(name="My Favorite Energy Drinks")

### Creating a class to house each of our energy drinks and their nutrition facts

In [654]:
class energyDrink:
    drinks = []
    def __init__(self, name, serving, caffeine, calories, sugar, b12, cost, model):
        self.name = name
        self.serv = serving
        self.caf = caffeine/serving
        self.cal = calories/serving
        self.sug = sugar/serving
        self.b12 = (2.4 * (100/b12))/serving # converting percentage to micrograms per fluid ounce
        # 2.4 mcg is the daily recommendation of vitamin B12 for those 19 years or older according to the National Institutes of Health
        self.cost = cost/serving
        self.var = model.continuous_var(name=f'Ounces of {name}')
        energyDrink.drinks.append(self)

    def varName(self):
        return f'Fluid ounces of {self.name}'

    def describe(self):
        print(f'{self.name} stats per fluid ounce:\n\t${self.cost:.2f}\n\t{self.caf:.3f}mg of caffeine\n\t{self.cal:.3f} calories\n\t{self.sug:.3f} grams of sugar\n\t{self.b12:.3f} micrograms of vitamin B12')

### Creating the Drink Objects
**Links:**
- [Peach Redbull]('https://a.co/d/3SlTlgr')
- [Pipeline Punch Monster]('https://a.co/d/2R8Gc4o')
- [Celcius Cola]('https://a.co/d/dnUJ96j')

In [657]:
R = energyDrink('Peach Redbull',
                serving = 8.4, # In fluid ounces
                caffeine = 80, # In milligrams
                calories = 110,
                sugar = 26,
                b12 = 80, # In percent of daily value
                cost = 38.99/24, # Cost derived from a 24 pack off of Amazon
                model = energyModel)

M = energyDrink('Pipeline Punch Monster',
                serving = 16,
                caffeine = 160,
                calories = 190,
                sugar = 46,
                b12 = 490,
                cost = 28.36/15, # Cost derived from 15 pack off of Amazon 
                model = energyModel)

C = energyDrink('Celcius Cola',
                serving = 12,
                caffeine = 200,
                calories = 10,
                sugar = 0,
                b12 = 250,
                cost = 22.98/12, # Cost derived from a 12 pack off of Amazon
                model = energyModel)

[drink.describe() for drink in energyDrink.drinks] # Using my describe method on all the drinks

Peach Redbull stats per fluid ounce:
	$0.19
	9.524mg of caffeine
	13.095 calories
	3.095 grams of sugar
	0.357 micrograms of vitamin B12
Pipeline Punch Monster stats per fluid ounce:
	$0.12
	10.000mg of caffeine
	11.875 calories
	2.875 grams of sugar
	0.031 micrograms of vitamin B12
Celcius Cola stats per fluid ounce:
	$0.16
	16.667mg of caffeine
	0.833 calories
	0.000 grams of sugar
	0.080 micrograms of vitamin B12


[None, None, None]

### Scenario 1: Getting 200mg of caffeine and the recommended amount of B12

In [666]:
energyModel.name = "My Favorite Energy Drinks: Scenario 1"
energyModel.remove_constraints(energyModel.iter_constraints()) # Removing constraints from other scenarios
constr1 = energyModel.add_constraint(energyModel.sum([drink.caf*drink.var for drink in energyDrink.drinks])<=400) # Can't have more than 400mg of caffeine
constr2 = energyModel.add_constraint(energyModel.sum([drink.caf*drink.var for drink in energyDrink.drinks])>=200) # I want to consume at least 200mg of caffeine
constr3 = energyModel.add_constraint(energyModel.sum([drink.b12*drink.var for drink in energyDrink.drinks])>=2.4) # I want the daily recommended amount of vitamin B12

energyModel.minimize(R.cost*R.var + M.cost*M.var + C.cost*C.var) # Minimizing for cost

### Scenario 2: Getting 300mg of caffeine but under the recommended sugar intake

In [620]:
energyModel.name = "My Favorite Energy Drinks: Scenario 2"
energyModel.remove_constraints(energyModel.iter_constraints()) # Removing constraints from other scenarios
constr1 = energyModel.add_constraint(energyModel.sum([drink.caf*drink.var for drink in energyDrink.drinks])<=400) # Can't have more than 400mg of caffeine
constr2 = energyModel.add_constraint(energyModel.sum([drink.caf*drink.var for drink in energyDrink.drinks])>=300) # I want to consume at least 200mg of caffeine
constr3 = energyModel.add_constraint(energyModel.sum([drink.sug*drink.var for drink in energyDrink.drinks])<=36) # Max recommended sugar intake for men

energyModel.minimize(R.cost*R.var + M.cost*M.var + C.cost*C.var) # Minimizing for cost

### Scenario 3: Getting as much vitamin B12 within a 6 dollar budget

In [660]:
energyModel.name = "My Favorite Energy Drinks: Scenario 3"
energyModel.remove_constraints(energyModel.iter_constraints()) # Removing constraints from other scenarios
constr1 = energyModel.add_constraint(energyModel.sum([drink.cost*drink.var for drink in energyDrink.drinks])<=6) # Can't have more than 400mg of caffeine
constr2 = energyModel.add_constraint(energyModel.sum([drink.cal*drink.var for drink in energyDrink.drinks])>=300) # I want to consume at least 200mg of caffeine

energyModel.maximize(R.b12*R.var + M.b12*M.var + C.b12*C.var) # Maximizing for B12

### Scenario 4: I want EXACTLY 347mg of caffeine AND maximum calories

In [642]:
energyModel.name = "My Favorite Energy Drinks: Scenario 4"
energyModel.remove_constraints(energyModel.iter_constraints()) # Removing constraints from other scenarios
constr1 = energyModel.add_constraint(energyModel.sum([drink.caf*drink.var for drink in energyDrink.drinks])==347)

energyModel.maximize(R.cal*R.var + M.cal*M.var + C.cal*C.var) # Maximizing for calories

### Solve the Selected Scenario

In [668]:
energyModel.print_information()

Model: My Favorite Energy Drinks: Scenario 1
 - number of variables: 3
   - binary=0, integer=0, continuous=3
 - number of constraints: 3
   - linear=3
 - parameters: defaults
 - objective: minimize
 - problem type is: LP


In [670]:
sol = energyModel.solve()
sol.display()

solution for: My Favorite Energy Drinks: Scenario 1
objective: 2.388
status: OPTIMAL_SOLUTION(2)
Ounces of Peach Redbull = 4.624
Ounces of Celcius Cola = 9.358
